# Pairwise correlation for robust feature selection

* Feature selection is important to remove redundant features, reduce model complexity, and avoid overfitting
* Some techniques focus on removal of features based on individual properties
* Pairwise correlation examines the connection between features and decides if they are worth keeping